#### Importation des bibliothèques nécessaires :

In [1]:
!pip install pandas sqlalchemy psycopg2-binary

In [2]:
import pandas as pd
import os
from sqlalchemy import create_engine

#### Configuration de la connexion à la base de données :

In [3]:
# Lire les variables d'environnement
db_host = os.getenv('DB_HOST')
db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

connection_string = f'postgresql://{db_user}:{db_password}@{db_host}/{db_name}'
engine = create_engine(connection_string)

#### Chargement des tables :

In [4]:
orders = pd.read_sql('SELECT * FROM orders', engine)
# Supprimer la colonne 'order_serial_id'
orders = orders.drop(columns=['order_serial_id'])

customers = pd.read_sql('SELECT * FROM customers', engine)
products = pd.read_sql('SELECT * FROM product', engine)
sales_team = pd.read_sql('SELECT * FROM sales_team', engine)
locations = pd.read_sql('SELECT * FROM location', engine)

#### Jointure des tables :

In [5]:
# Jointure de orders avec customers
orders_customers = orders.merge(customers, on='customer_id', how='left')

# Jointure de orders_customers avec product
orders_customers_products = orders_customers.merge(products, on='product_id', how='left')

# Jointure de orders_customers_products avec sales_team
orders_customers_products_sales = orders_customers_products.merge(sales_team, on='sales_rep', how='left')

# Jointure de orders_customers_products_sales avec location
complete_data = orders_customers_products_sales.merge(locations, on='location_id', how='left')

#### Sauvegarde des données reconstituées dans un fichier CSV :

In [6]:
complete_data.to_csv('data/reconstructed_SuperStoreData.csv', index=False)